In [157]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import csv

import torch
import captum
from captum.attr import IntegratedGradients
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


# Set device (GPU if available)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [158]:
# import argparse


# parser = argparse.ArgumentParser(description="Run script with optional CU and DU values.")
# parser.add_argument("--cu", type=int, default=0, help="CU value (default: 0)")
# parser.add_argument("--du", type=int, default=0, help="DU value (default: 0)")

# args = parser.parse_args()

# CU, DU = args.cu, args.du

CU = 0
DU = 0

# Load dataset
dataset = pd.read_csv(f'dataset_srscu{CU}_srsdu{DU}.csv')
# dataset = dataset[:int(0.01*len(dataset))]

dataset.index = dataset['Timestamp']
dataset = dataset.drop(columns=['Timestamp'])

dataset.head()

,"((node_memory_MemTotal_bytes{instance=""node-exporter:9100"", job=""node""} - node_memory_MemFree_bytes{instance=""node-exporter:9100"", job=""node""}) / node_memory_MemTotal_bytes{instance=""node-exporter:9100"", job=""node""}) * 100","((node_memory_SwapTotal_bytes{instance=""node-exporter:9100"",job=""node""} - node_memory_SwapFree_bytes{instance=""node-exporter:9100"",job=""node""}) / (node_memory_SwapTotal_bytes{instance=""node-exporter:9100"",job=""node""})) * 100","((sum(container_memory_usage_bytes{name=""srscu0"",instance=""cadvisor:8080""}) by (instance) - sum(container_memory_cache{name=""srscu0"",instance=""cadvisor:8080""}) by (instance)) / sum(machine_memory_bytes{instance=""cadvisor:8080""}) by (instance)) * 100","((sum(container_memory_usage_bytes{name=""srsdu0"",instance=""cadvisor:8080""}) by (instance) - sum(container_memory_cache{name=""srsdu0"",instance=""cadvisor:8080""}) by (instance)) / sum(machine_memory_bytes{instance=""cadvisor:8080""}) by (instance)) * 100","(1 - (node_memory_MemAvailable_bytes{instance=""node-exporter:9100"", job=""node""} / node_memory_MemTotal_bytes{instance=""node-exporter:9100"", job=""node""})) * 100","(node_memory_SwapTotal_bytes{instance=""node-exporter:9100"",job=""node""} - node_memory_SwapFree_bytes{instance=""node-exporter:9100"",job=""node""})","100 * (1 - avg(rate(node_cpu_seconds_total{mode=""idle"", instance=""node-exporter:9100""}[10s])))","100 - ((node_filesystem_avail_bytes{instance=""node-exporter:9100"",job=""node"",device!~'rootfs'} * 100) / node_filesystem_size_bytes{instance=""node-exporter:9100"",job=""node"",device!~'rootfs'})","100 - ((node_filesystem_avail_bytes{instance=""node-exporter:9100"",job=""node"",mountpoint=""/"",fstype!=""rootfs""} * 100) / node_filesystem_size_bytes{instance=""node-exporter:9100"",job=""node"",mountpoint=""/"",fstype!=""rootfs""})",PCI-1_RNTI-4601_bsr,...,"sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode!='idle',mode!='user',mode!='system',mode!='iowait',mode!='irq',mode!='softirq'}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=""idle""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=""iowait""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=""irq""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=""nice""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=""softirq""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=""steal""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=""system""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=""user""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node""}) by (cpu)))","sum(irate(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""node"", mode=~"".*irq""}[10s])) / scalar(count(count(node_cpu_seconds_total{instance=""node-exporter:9100"",job=""nod

In [159]:
features = dataset.drop(columns=[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"])
targets = dataset[[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"]]

In [160]:
true_labels = targets[[f"srscu{CU}_stressType", f"srsdu{DU}_stressType"]].values
output = [] 
for val in true_labels:
    if val[0] != 0 or val[1] != 0:
        output.append(1)
    else:
        output.append(0)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

index = int(0.8*len(features))
# Reassign train/test sets properly for classifier training
X_train, X_test = features[:index], features[index:]  # features was re-assigned above
y_train, y_test = output[:index], output[index:]    # output was re-assigned above

# Initialize and train the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42,  class_weight='balanced')
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred))

# Print confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.97      0.98       300

    accuracy                           0.97       300
   macro avg       0.50      0.48      0.49       300
weighted avg       1.00      0.97      0.98       300

Confusion Matrix:
[[  0   0]
 [  9 291]]


/home/michael/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/michael/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/michael/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [161]:
# **Data Preprocessing**
# Handle missing values
features = features.apply(lambda x: x.fillna(0) if x.isna().all() else x)


threshold = 0.6 * len(features)
features = features.loc[:, ~features.columns.duplicated()]  # Remove duplicates

for col in features.columns:
    nan_count = features[col].isna().sum()
    if int(nan_count) > threshold:  # Explicit scalar conversion
        mode_value = features[col].mode().iloc[0] if not features[col].mode().empty else 0
        features[col].fillna(mode_value, inplace=True)

numeric_cols = features.select_dtypes(include=[np.number]).columns
features[numeric_cols] = features[numeric_cols].fillna(features[numeric_cols].mean())

In [162]:
import joblib
import pandas as pd

# Load the saved scaler
scaler = joblib.load('minmax_scaler.pkl')
pca = joblib.load('pca_model.pkl')

# Scale the new features using the same scaler
features_scaled = scaler.transform(features)


# Apply PCA transformation using the loaded PCA model
features_pca = pca.transform(features_scaled)


# Convert to DataFrame
features_pca = pd.DataFrame(features_pca, columns=[f'Reduced-{i+1}' for i in range(pca.n_components_)])
features_pca.index = features.index

/home/michael/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


In [163]:
LOOK_BACK, LOOK_FORWARD = 60, 5


def create_sequences(features_pca, targets, look_back, look_forward, CU, DU):
    """
    Create LSTM-ready sequences from a DataFrame.
    """
    pca_features_input, pca_features_output, input_targets, output_targets = [], [], [], []

    for i in range(len(features_pca) - look_back - look_forward + 1):
        pca_features_input.append(features_pca.iloc[i:(i + look_back)].values)
        pca_features_output.append(features_pca.iloc[(i + look_back):i + look_back + look_forward ].values)

        input_targets.append(targets.iloc[i:(i + look_back)].values)
        output_targets.append(targets.iloc[(i + look_back):i + look_back + look_forward ])  # assuming this is a scalar/class label

    return pca_features_input, pca_features_output, input_targets, output_targets


pca_features_input , pca_features_output, input_targets, output_targets = create_sequences(features_pca, targets, LOOK_BACK, LOOK_FORWARD, CU, DU)

print(f"Input shape: {np.array(pca_features_input).shape}")
print(f"Output shape: {np.array(output_targets).shape}")


Input shape: (1435, 60, 9)
Output shape: (1435, 5, 4)


In [164]:
# **Define LSTM Model**
class LSTMForecaster(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMForecaster, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        return out

def autoregressive_forecast(model_path, initial_sequence, steps_ahead=5, device='cpu'):
    """
    Predict next `steps_ahead` steps using autoregressive LSTM inference.
    
    Parameters:
        model_path (str): Path to the saved LSTM model (.pth).
        initial_sequence (np.ndarray or torch.Tensor): Input of shape (60, M).
        steps_ahead (int): Number of future steps to predict.
        device (str): 'cuda' or 'cpu'.
    
    Returns:
        predicted_sequence (torch.Tensor): Tensor of shape (steps_ahead, M) with predictions.
    """
    # Load the model
    model = torch.load(model_path, map_location=device)
    model.eval()

    # Ensure input is a torch tensor of shape (1, 60, M)
    if isinstance(initial_sequence, np.ndarray):
        initial_sequence = torch.tensor(initial_sequence, dtype=torch.float32)
    
    if len(initial_sequence.shape) == 2:
        initial_sequence = initial_sequence.unsqueeze(0)

   
    
    initial_sequence = initial_sequence.to(device)
    
    generated_sequence = []

    current_sequence = initial_sequence.clone()

    for _ in range(steps_ahead):
        with torch.no_grad():
            # Predict next step
            next_step = model(current_sequence)  # (1, M)
        
        generated_sequence.append(next_step.squeeze(0))  # (M,)
        
        # Prepare input for next step by appending prediction and removing oldest timestep
        next_step_expanded = next_step.unsqueeze(1)  # (1, 1, M)
        current_sequence = torch.cat((current_sequence[:, 1:, :], next_step_expanded), dim=1)  # (1, 60, M)

    return torch.stack(generated_sequence)  # (steps_ahead, M)



model_path = 'lstm_models/lstm_fold1_RMSprop.pt'
full_sequences_with_predictions = []

for i, sequence in enumerate(pca_features_input):
    predicted_future = autoregressive_forecast(model_path, sequence, steps_ahead=LOOK_FORWARD)  # (5, M)
    
    # Convert input sequence and prediction to tensors if not already
    if isinstance(sequence, np.ndarray):
        sequence_tensor = torch.tensor(sequence, dtype=torch.float32)
    else:
        sequence_tensor = sequence

    # Concatenate original 60 input samples with 5 predicted ones → (65, M)
    full_sequence = torch.cat([sequence_tensor, predicted_future.cpu()], dim=0)
    full_sequences_with_predictions.append(full_sequence.numpy())

# Final shape: (num_sequences, 65, M)
full_sequences_with_predictions = np.array(full_sequences_with_predictions)

print("Shape of sequences with input + predictions:", full_sequences_with_predictions.shape)



/tmp/ipykernel_22544/428938438.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


Shape of sequences with input + predictions: (1435, 65, 9)


In [165]:
# Flatten from (num_sequences, 65, reduced_dim) → (num_sequences * 65, reduced_dim)
full_sequences_with_predictions = full_sequences_with_predictions.reshape(-1, full_sequences_with_predictions.shape[-1])
print("Shape before inverse PCA:", full_sequences_with_predictions.shape)

# Inverse PCA transformation → (num_sequences * 65, original_feature_dim)
full_sequences_with_predictions = pca.inverse_transform(full_sequences_with_predictions)
print("Shape after inverse PCA:", full_sequences_with_predictions.shape)

# Reshape back to (num_sequences, 65, original_feature_dim)
original_feature_dim = pca.inverse_transform(np.zeros((1, pca.n_components_))).shape[1]

full_sequences_with_predictions = full_sequences_with_predictions.reshape(
    -1, LOOK_BACK + LOOK_FORWARD, original_feature_dim
)
print("Final shape after inverse PCA:", full_sequences_with_predictions.shape)


Shape before inverse PCA: (93275, 9)
Shape after inverse PCA: (93275, 290)
Final shape after inverse PCA: (1435, 65, 290)


In [166]:
# === JVGAN COMPONENTS ===
class JVGANEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(JVGANEncoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, 128, batch_first=True)
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_logvar = nn.Linear(128, latent_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h = h_n.squeeze(0)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        std = torch.exp(0.5 * logvar)
        z = mu + std * torch.randn_like(std)
        return z, mu, logvar

class JVGANDecoder(nn.Module):
    def __init__(self, latent_dim, input_dim, sequence_length=65):
        super(JVGANDecoder, self).__init__()
        self.latent_to_hidden = nn.Linear(latent_dim, 128)
        self.lstm = nn.LSTM(128, 128, batch_first=True)
        self.output_layer = nn.Linear(128, input_dim)
        self.sequence_length = sequence_length

    def forward(self, z):
        h0 = self.latent_to_hidden(z).unsqueeze(1).repeat(1, self.sequence_length, 1)
        lstm_out, _ = self.lstm(h0)
        return self.output_layer(lstm_out)

class JVGANDiscriminator(nn.Module):
    def __init__(self, input_dim):
        super(JVGANDiscriminator, self).__init__()
        self.lstm = nn.LSTM(input_dim, 128, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.classifier(h_n.squeeze(0))

class JVGAN(nn.Module):
    def __init__(self, input_dim, latent_dim=32, sequence_length=65):
        super(JVGAN, self).__init__()
        self.encoder = JVGANEncoder(input_dim, latent_dim)
        self.decoder = JVGANDecoder(latent_dim, input_dim, sequence_length)
        self.discriminator = JVGANDiscriminator(input_dim)

    def forward(self, x):
        z, mu, logvar = self.encoder(x)
        x_recon = self.decoder(z)
        d_real = self.discriminator(x)
        d_fake = self.discriminator(x_recon.detach())
        return x_recon, d_real, d_fake, mu, logvar

import torch.nn.functional as F


def detect_anomaly(x, model, threshold):
    model.eval()
    with torch.no_grad():
        x_recon, _, _, _, _ = model(x)
        last_real = x[:, -1, :]
        last_recon = x_recon[:, -1, :]
        error = F.mse_loss(last_recon, last_real, reduction='none').mean(dim=1)
        return error > threshold, error




In [167]:
# load the JVGAN model
model_path = 'jvgan_models/jvgan_fold1.pt'
# testing of jvgan model
model = torch.load(model_path, map_location=device)
# Define the threshold for anomaly detection
threshold = 0.05  # Adjust based on your dataset and requirements

# Validation
model.eval()
is_anomaly, recon_errors = detect_anomaly(torch.tensor(full_sequences_with_predictions, dtype=torch.float32), model, threshold)
print(is_anomaly)


true_labels = [1 if (val.iloc[-1][f'srscu{CU}_stressType'] != 0 or val.iloc[-1][f'srsdu{DU}_stressType'] != 0) else 0 for val in output_targets]
# Calculate accuracy
# accuracy = np.mean(is_anomaly.numpy() == true_labels)
predicted_labels = []
for val in is_anomaly.numpy():
    if(val==False):
        predicted_labels.append(0)
    else:
        predicted_labels.append(1)
accuracy = np.mean(predicted_labels == true_labels)

print(f"Accuracy: {accuracy * 100:.2f}%")
# Calculate F1 score
f1 = f1_score(true_labels, is_anomaly.numpy())
print(f"F1 Score: {f1:.2f}")
# Print classification report
print(classification_report(true_labels, is_anomaly.numpy(), target_names=['Normal', 'Anomalous']))
# Print confusion matrix
conf_matrix = confusion_matrix(true_labels, is_anomaly.numpy())
print("Confusion Matrix:")
print(conf_matrix)
# Save the results to a CSV file
results_df = pd.DataFrame({
    'True Label': true_labels,
    'Predicted Anomaly': is_anomaly.numpy(),
    'Reconstruction Error': recon_errors.numpy()
})
results_df.to_csv(f'results_srscu{CU}_srsdu{DU}.csv', index=False)

/tmp/ipykernel_22544/1901275868.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


tensor([False, False, False,  ..., False, False, False])
Accuracy: 0.00%
F1 Score: 0.08
              precision    recall  f1-score   support

      Normal       0.17      0.90      0.28       252
   Anomalous       0.69      0.04      0.08      1183

    accuracy                           0.20      1435
   macro avg       0.43      0.47      0.18      1435
weighted avg       0.60      0.20      0.12      1435

Confusion Matrix:
[[ 228   24]
 [1130   53]]


In [168]:
# original_feature_names = features.columns.tolist()  # after inverse_transform

# import shap
# import torch
# import numpy as np
# import torch.nn as nn

# # === Step 1: Wrap Decoder for SHAP ===
# class LastStepDecoder(nn.Module):
#     def __init__(self, jvgan_model):
#         super().__init__()
#         self.encoder = jvgan_model.encoder
#         self.decoder = jvgan_model.decoder

#     def forward(self, x):
#         z, _, _ = self.encoder(x)
#         x_recon = self.decoder(z)
#         return x_recon[:, -1, :]  # Only last timestep reconstruction

# # === Step 2: Select Anomalous Samples ===
# anomaly_indices = torch.nonzero(is_anomaly).squeeze().cpu().numpy()
# if anomaly_indices.ndim == 0:
#     anomaly_indices = anomaly_indices.reshape(1)

# # Limit to a small number for SHAP (e.g., first 5–10 anomalies)
# max_explain = min(10, len(anomaly_indices))
# samples_to_explain = torch.tensor(full_sequences_with_predictions[anomaly_indices[:max_explain]], dtype=torch.float32).to(device)
# print("Initialization started")

# # === Step 3: Wrap Model & Initialize SHAP Explainer ===
# wrapped_decoder = LastStepDecoder(model).to(device)

# explainer = shap.GradientExplainer(wrapped_decoder, samples_to_explain)
# # Time taken for initialization
# print("Initialization done. SHAP Explainer ready to compute values.")

# # === Step 4: Compute SHAP Values ===
# shap_values = explainer.shap_values(samples_to_explain)  # List with one element: (N, M)
# print("computation of SHAP values done.")

# # === Step 5: Display Top 10 Features ===
# for i, shap_val in enumerate(shap_values[0]):
#     top_10_idx = np.argsort(np.abs(shap_val))[-10:][::-1]
#     print(f"\nTop 10 impacting features for Anomalous Sample #{i+1}:")
#     for rank, idx in enumerate(top_10_idx, start=1):
#         print(f"{rank}. {original_feature_names[idx]} | SHAP value: {shap_val[idx]:.4f}")
# # === Step 6: Visualize SHAP Values ===
# shap.summary_plot(shap_values[0], samples_to_explain.cpu().numpy(), feature_names=original_feature_names, max_display=10)
# # Save the SHAP values to a CSV file
# shap_df = pd.DataFrame(shap_values[0], columns=original_feature_names)
# shap_df.to_csv(f'shap_values_srscu{CU}_srsdu{DU}.csv', index=False)




In [169]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from captum.attr import IntegratedGradients

# === Step 1: Extract original feature names ===
original_feature_names = features.columns.tolist()

# === Step 2: Define wrapper model returning last timestep ===
class LastStepDecoder(nn.Module):
    def __init__(self, jvgan_model):
        super().__init__()
        self.encoder = jvgan_model.encoder
        self.decoder = jvgan_model.decoder

    def forward(self, x):
        z, _, _ = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon[:, -1, :]  # (batch_size, M)

# === Step 3: Select Anomalous Samples ===
anomaly_indices = torch.nonzero(is_anomaly).squeeze().cpu().numpy()
if anomaly_indices.ndim == 0:
    anomaly_indices = anomaly_indices.reshape(1)

max_explain = min(100, len(anomaly_indices))
samples_to_explain = torch.tensor(full_sequences_with_predictions[anomaly_indices[:max_explain]],
                                  dtype=torch.float32).to(device)

# === Step 4: Setup model and IG explainer ===
wrapped_decoder = LastStepDecoder(model).to(device).eval()
ig = IntegratedGradients(wrapped_decoder)
baseline = torch.zeros_like(samples_to_explain)

# === Step 5: Compute attributions ===
all_attributions = []

for i in range(samples_to_explain.size(0)):
    input_sample = samples_to_explain[i].unsqueeze(0)         # (1, 65, M)
    baseline_sample = baseline[i].unsqueeze(0)                # (1, 65, M)

    with torch.no_grad():
        recon = wrapped_decoder(input_sample)                 # (1, M)
        original = input_sample[:, -1, :]                     # (1, M)
        recon_error = torch.abs(recon - original)             # (1, M)
        target_feature = torch.argmax(recon_error).item()     # int

    # Compute IG for the selected output index
    attr, _ = ig.attribute(input_sample, baselines=baseline_sample,
                           target=target_feature,
                           return_convergence_delta=True)

    # Attribution shape: (1, 65, M), extract last step
    attr_last_step = attr.squeeze(0)[-1].detach().cpu().numpy()
    all_attributions.append(attr_last_step)

    # === Step 6: Print Top 10 Features ===
    top_10_idx = np.argsort(np.abs(attr_last_step))[-10:][::-1]
    print(f"\nTop 10 impacting features using Integrated Gradients for Anomalous Sample #{i+1}:")
    for rank, idx in enumerate(top_10_idx, 1):
        print(f"{rank}. {original_feature_names[idx]} | IG Value: {attr_last_step[idx]:.4f}")

# === Step 7: Save IG attributions to CSV ===
ig_df = pd.DataFrame(all_attributions, columns=original_feature_names)
ig_df.to_csv(f'integrated_gradients_srscu{CU}_srsdu{DU}.csv', index=False)



Top 10 impacting features using Integrated Gradients for Anomalous Sample #1:
1. irate(node_netstat_Tcp_RetransSegs{instance="node-exporter:9100",job="node"}[10s]) | IG Value: 0.0005
2. irate(node_netstat_TcpExt_SyncookiesFailed{instance="node-exporter:9100",job="node"}[10s]) | IG Value: -0.0005
3. node_sockstat_UDP_mem{instance="node-exporter:9100",job="node"} | IG Value: -0.0005
4. node_memory_HugePages_Free{instance="node-exporter:9100",job="node"} | IG Value: 0.0004
5. sum by(instance) (irate(node_cpu_guest_seconds_total{instance="node-exporter:9100",job="node", mode="nice"}[1m])) / on(instance) group_left sum by (instance)((irate(node_cpu_seconds_total{instance="node-exporter:9100",job="node"}[1m]))) | IG Value: -0.0004
6. sum by(instance) (irate(node_cpu_guest_seconds_total{instance="node-exporter:9100",job="node", mode="user"}[1m])) / on(instance) group_left sum by (instance)((irate(node_cpu_seconds_total{instance="node-exporter:9100",job="node"}[1m]))) | IG Value: -0.0004
7. n

In [170]:
# import networkx as nx
# import os
# import pickle

# # === Step 8: Load causal graph ===
# causal_graph_path = "causal_graph.gpickle"
# if not os.path.exists(causal_graph_path):
#     raise FileNotFoundError("Causal graph not found! Please run the Granger causality code first.")

# with open("causal_graph.gpickle", "rb") as f:
#     G = pickle.load(f)


# # === Step 9: Root cause tracing function ===
# def find_root_causes(graph, feature):
#     """
#     For a given feature, walk backward in the graph to find root causes
#     (nodes with no incoming edges that can reach this node).
#     """
#     root_causes = set()

#     def dfs(node, visited):
#         if node in visited:
#             return
#         visited.add(node)
#         preds = list(graph.predecessors(node))
#         if not preds:
#             root_causes.add(node)
#         else:
#             for pred in preds:
#                 dfs(pred, visited)

#     dfs(feature, set())
#     return list(root_causes)

# # === Step 10: Extract top IG features and rank root causes ===
# ranked_root_cause_rows = []

# for i, row in ig_df.iterrows():
#     anomaly_id = f"anomaly_{i+1:03d}"
#     abs_values = np.abs(row.values)
#     top_10_idx = np.argsort(abs_values)[-10:][::-1]
#     top_features = [original_feature_names[idx] for idx in top_10_idx]

#     root_cause_counter = {}

#     for f in top_features:
#         if f in G.nodes:
#             root_causes = find_root_causes(G, f)
#             for root in root_causes:
#                 root_cause_counter[root] = root_cause_counter.get(root, 0) + 1
#         else:
#             continue

#     if not root_cause_counter:
#         ranked_root_cause_rows.append({
#             "anomaly_id": anomaly_id,
#             "root_cause": "None",
#             "rank": 1,
#             "count": 0
#         })
#         continue

#     # Rank by frequency (descending)
#     sorted_roots = sorted(root_cause_counter.items(), key=lambda x: -x[1])
#     for rank, (rc, count) in enumerate(sorted_roots, 1):
#         ranked_root_cause_rows.append({
#             "anomaly_id": anomaly_id,
#             "root_cause": rc,
#             "rank": rank,
#             "count": count
#         })

# # === Step 11: Save ranked root causes ===
# ranked_df = pd.DataFrame(ranked_root_cause_rows)
# ranked_df.to_csv(f'ranked_root_causes_srscu{CU}_srsdu{DU}.csv', index=False)
# print(f"\nSaved ranked root causes to: ranked_root_causes_srscu{CU}_srsdu{DU}.csv")


In [171]:
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import os

# === Step 1: Load the full causal graph ===
with open("causal_graph.gpickle", "rb") as f:
    full_graph = pickle.load(f)

# === Step 2: Define function to get subgraph and root causes ===
def get_subgraph_and_roots(graph, top_features):
    sub_G = graph.subgraph(top_features).copy()
    root_nodes = [n for n in sub_G.nodes if sub_G.in_degree(n) == 0]
    return sub_G, root_nodes

# === Step 3: Analyze IG and causal graph for each anomaly ===
anomaly_subgraphs = {}
root_cause_records = []

for i, row in ig_df.iterrows():
    anomaly_id = f"anomaly_{i+1:03d}"
    
    # Step 3.1: Top 10 features by IG magnitude
    abs_values = np.abs(row.values)
    top_10_idx = np.argsort(abs_values)[-10:][::-1]
    top_features = [original_feature_names[idx] for idx in top_10_idx]

    # Step 3.2: Build causal subgraph and get root causes
    subgraph, roots = get_subgraph_and_roots(full_graph, top_features)
    anomaly_subgraphs[anomaly_id] = subgraph

    if roots:
        for rank, r in enumerate(roots, 1):
            root_cause_records.append({
                "anomaly_id": anomaly_id,
                "root_cause": r,
                "rank": rank
            })
    else:
        root_cause_records.append({
            "anomaly_id": anomaly_id,
            "root_cause": "None",
            "rank": 1
        })

# === Step 4: Save root causes and optionally subgraphs ===
root_df = pd.DataFrame(root_cause_records)
root_df.to_csv(f'root_causes_top10_srscu{CU}_srsdu{DU}.csv', index=False)
print(f"Saved root causes per anomaly to: root_causes_top10_srscu{CU}_srsdu{DU}.csv")

# Optional: Save each subgraph individually as a .gpickle
save_dir = "anomaly_subgraphs"
os.makedirs(save_dir, exist_ok=True)
for anomaly_id, sub_G in anomaly_subgraphs.items():
    with open(os.path.join(save_dir, f"{anomaly_id}_subgraph.gpickle"), "wb") as f:
        pickle.dump(sub_G, f)


Saved root causes per anomaly to: root_causes_top10_srscu0_srsdu0.csv
